In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pylab as pl
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))


In [ ]:
import numpy as np # linear algebra
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-2/train.csv')
test = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-2/test.csv')

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.sample(3)

In [ ]:
train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])

In [ ]:
train.info()

In [ ]:
train['Date'] = train['Date'].astype('int64')
test['Date'] = test['Date'].astype('int64')

In [ ]:
train.info()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
def FunLabelEncoder(df):
    for c in df.columns:
        if df.dtypes[c] == object:
            le.fit(df[c].astype(str))
            df[c] = le.transform(df[c].astype(str))
    return df

In [ ]:
train = FunLabelEncoder(train)
train.info()
train.iloc[235:300,:]

In [ ]:
test = FunLabelEncoder(test)
test.info()

In [ ]:
train.iloc[:,:-2].sample(3)

In [ ]:
X = train.iloc[:,:-2]
print(X.shape)
X.sample(3)

In [ ]:
Y = train.iloc[:,-2:]
print(Y.shape)
Y.sample(3)

In [ ]:
from sklearn.model_selection import train_test_split 
trainX , valX, trainY, valY = train_test_split(X, Y, random_state=1)

In [ ]:
y1Train = trainY.iloc[:,0]
print(y1Train.shape)
y1Train.sample(3)

In [ ]:
y2Train = trainY.iloc[:,1]
y2Train.sample(3)

In [ ]:
y1Val = valY.iloc[:,0]
y1Val.sample(3)

In [ ]:
y2Val = valY.iloc[:,1]
y2Val.sample(3)

In [ ]:
print(trainX.shape)
trainX.sample(3)

In [ ]:
print(trainX.info())

In [ ]:
trainX.iloc[:,1:].sample(3)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
lrModel1 = DecisionTreeRegressor(random_state = 27)
%time lrModel1.fit(trainX.iloc[:,1:], y1Train)

In [ ]:
%time y1Pred = lrModel1.predict(valX.iloc[:,1:])
print(y1Pred[:,])

In [ ]:
from sklearn.metrics import mean_absolute_error

print("Accuracy in train set : ", lrModel1.score(trainX.iloc[:,1:], y1Train))
print("RMSE : ", mean_absolute_error(y1Val, y1Pred)**(0.5))

In [ ]:
lrModel2 = DecisionTreeRegressor(random_state = 27)
%time lrModel2.fit(trainX.iloc[:,1:], y2Train)

%time y2Pred = lrModel2.predict(valX.iloc[:,1:])

print("Accuracy in train set : ", lrModel2.score(trainX.iloc[:,1:], y2Train))
print("RMSE : ", mean_absolute_error(y2Val, y2Pred)**(0.5))

In [ ]:
print(test.shape)
test.sample(3)

In [ ]:
test.iloc[:,1:].sample(3)

In [ ]:
%time finalPred1 = lrModel1.predict(test.iloc[:,1:])
print(finalPred1[:,])

In [ ]:
%time finalPred2 = lrModel2.predict(test.iloc[:,1:])
print(finalPred2[:,])

In [ ]:
outputFile = pd.DataFrame({"ForecastId": test.ForecastId,
                           "ConfirmedCases": (finalPred1+0.5).astype('int'),
                           "Fatalities": (finalPred2+0.5).astype('int')})

In [ ]:
outputFile.sample(3)

In [ ]:
outputFile.to_csv("submission.csv", index=False)